<a href="https://colab.research.google.com/github/LeonardoGomesG/Codifica-oZipIme/blob/master/ggl_kpi_valor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Budget Manager -  Cálculo do Valor da Métrica de Otimização

O objetivo desta etapa é calcular o Valor da Métrica de Otimização escolhida no processo anterior. \\
O valor de todos os kpis das campanhas serão calculados de acordo com uma janela fixa de dias (futuramente uma janela móvel) a partir do dia atual, dada pelo parametro METRIC_WINDOW definido abaixo. \\
Será calculado também o valor do coeficiente usado na otimização do simplex, definindo se quer-se minimizar ou maximizar a Métrica de Otimização.




### Configuração do notebook

Utilizaremos as biblitecas pandas e numpy para trabalhar com o dataset. Para o carregamento e consulta dos dados, utilizaremos a biblioteca do google cloud. Abaixo, iremos configurar os parâmetros globais do algoritmo.

In [14]:
import os
import warnings
import math

from datetime import datetime, timedelta

warnings.filterwarnings('ignore', category=FutureWarning)

import pandas as pd
import numpy as np

In [15]:
PARTITION_DATE = locals().get('ds', (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d"))
PARTITION_DATE_NODASH = locals().get('ds_nodash', (datetime.today() - timedelta(days=1)).strftime("%Y%m%d"))

GOOGLE_AUTH_CREDENTIALS_FILE = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS', '/content/credentials.json')

GOOGLE_BIGQUERY_PROJECT = 'pareto-bi'
GOOGLE_BIGQUERY_DATASET = locals().get('gbq_dataset', 'BUDGET_MANAGER_TEST')
GOOGLE_BIGQUERY_DESTINATION_TABLE = f'{GOOGLE_BIGQUERY_PROJECT}.{GOOGLE_BIGQUERY_DATASET}.ggl_optimization_metric_value${PARTITION_DATE_NODASH}'

METRIC_WINDOW = 7
MW_PARAMETER = METRIC_WINDOW - 1

In [16]:
from google.cloud import bigquery
from google.cloud.bigquery import magics
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(GOOGLE_AUTH_CREDENTIALS_FILE)
magics.context.credentials = credentials

client = bigquery.Client(credentials=credentials, project=GOOGLE_BIGQUERY_PROJECT)

%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### Montagem do dataset

In [17]:
job_config = bigquery.QueryJobConfig(
  destination=GOOGLE_BIGQUERY_DESTINATION_TABLE,
  write_disposition="WRITE_TRUNCATE",
  create_disposition="CREATE_IF_NEEDED",
  schema_update_options=["ALLOW_FIELD_ADDITION", "ALLOW_FIELD_RELAXATION"],
  use_legacy_sql=False,
  time_partitioning=bigquery.table.TimePartitioning(field="day"),
)

query = client.query(f"""
WITH
###
# Realiza a soma das métricas brutas nos últimos @MOVING_AVARAGE_DAYS
###
campaign_performance_over_window AS (
  SELECT
    *,
    SUM(impressions)                           OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS impressions_over_window,
    SUM(clicks)                                OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS clicks_over_window,
    SUM(video_views)                           OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS video_views_over_window,
    SUM(cost)                                  OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS cost_over_window,
    SUM(conversions)                           OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS conversions_over_window,
    SUM(conversion_value_sales)                OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS conversion_value_sales_over_window,
    MAX(SAFE_DIVIDE(cost, conversions))        OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS max_cpa_over_window,
    MAX(SAFE_DIVIDE(cost, clicks))             OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS max_cpc_over_window,
    MAX(SAFE_DIVIDE(cost, impressions / 1000)) OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS max_cpm_over_window,
    MAX(SAFE_DIVIDE(cost, video_views))        OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS max_cpv_over_window,
    COUNT(*)                                   OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN {MW_PARAMETER} PRECEDING AND CURRENT ROW) AS num_days
  FROM
    `{GOOGLE_BIGQUERY_PROJECT}.{GOOGLE_BIGQUERY_DATASET}.ggl_campaign_performance`
  WHERE
    day >= DATE_SUB(DATE("{PARTITION_DATE}"), INTERVAL 14 DAY)
),

###
# Realiza o cálculo dos kpis nos últimos @MOVING_AVARAGE_DAYS
###
campaign_performance_with_kpis_over_window AS (
  SELECT
    *,
    SAFE_DIVIDE(cost_over_window, conversions_over_window) AS cpa_over_window,
    SAFE_DIVIDE(cost_over_window, clicks_over_window) AS cpc_over_window,
    SAFE_DIVIDE(cost_over_window, impressions_over_window / 1000) AS cpm_over_window,
    SAFE_DIVIDE(cost_over_window, video_views_over_window) AS cpv_over_window,
    SAFE_DIVIDE(conversion_value_sales_over_window, cost_over_window) AS roas_over_window
  FROM
    campaign_performance_over_window
),

###
# Realiza o cálculo do optimization_metric_value nos últimos @MOVING_AVARAGE_DAYS
# particionado por dia
# Caso optimization_metric_value seja nulo, 
# assumiremos que a campanha teve o pior resultado dentre as campanhas, 
# sendo de menor prioridade na optimização no simplex, ordenando pelo custo dentre as de valor nulo
###
campaign_performance_with_kpi_value_over_window AS (
  SELECT
    kd.day,
    kd.account_id,
    kd.account_name,
    kd.campaign_id,
    kd.campaign_name,
    kd.version,
    kd.pool_id,
    kd.pool_name,
    kd.objective_id,
    kd.objective_name,
    kd.objective_budget,
    kd.bid_strategy_type,
    kd.subobjective_name,
    kd.subobjective_budget,
    kd.subobjective_base_daily_budget,
    kd.subobjective_daily_budget,
    kd.subobjective_cost_share,
    kd.last_month_objective_cost,
    kd.last_month_subobjective_cost,
    kd.this_month_subobjective_cost,
    kd.optimization_metric,
    num_days,
    cpa_over_window,
    cpc_over_window,
    cpm_over_window,
    cpv_over_window,
    roas_over_window,
    CASE
        WHEN optimization_metric = "CPC" THEN IFNULL(cpc_over_window, IFNULL(max_cpc_over_window, 0) + cost_over_window)
        WHEN optimization_metric = "CPA" THEN IFNULL(cpa_over_window, IFNULL(max_cpa_over_window, 0) + cost_over_window)
        WHEN optimization_metric = "CPM" THEN IFNULL(cpm_over_window, IFNULL(max_cpm_over_window, 0) + cost_over_window)
        WHEN optimization_metric = "CPV" THEN IFNULL(cpv_over_window, IFNULL(max_cpv_over_window, 0) + cost_over_window)
        WHEN optimization_metric = "ROAS" THEN IF(IFNULL(roas_over_window, 0) = 0, IFNULL(SAFE_DIVIDE(1, cost_over_window), 0), roas_over_window)
    END optimization_metric_value,
    CASE
        WHEN optimization_metric = "CPC" THEN IF(cpc_over_window IS NULL, 'virtual', 'real')
        WHEN optimization_metric = "CPA" THEN IF(cpa_over_window IS NULL, 'virtual', 'real')
        WHEN optimization_metric = "CPM" THEN IF(cpm_over_window IS NULL, 'virtual', 'real')
        WHEN optimization_metric = "CPV" THEN IF(cpv_over_window IS NULL, 'virtual', 'real')
        WHEN optimization_metric = "ROAS" THEN IF(roas_over_window IS NULL, 'virtual', 'real')
    END optimization_metric_value_source,
  FROM 
    `{GOOGLE_BIGQUERY_PROJECT}.{GOOGLE_BIGQUERY_DATASET}.ggl_kpi_definition` as kd
  LEFT JOIN campaign_performance_with_kpis_over_window as kpiw
  USING (account_id, campaign_id, day)
  WHERE
    day = "{PARTITION_DATE}"
),

###
# Realiza o cálculo do optimization_metric_coeficient nos últimos @MOVING_AVARAGE_DAYS
###
campaign_performance_with_kpi_coeficient_over_window AS (
  SELECT
    *,
    UTIL.get_optimization_coeficient_ggl(optimization_metric, optimization_metric_value) as optimization_metric_coeficient
  FROM 
    campaign_performance_with_kpi_value_over_window
)

###
# Filtra as colunas necessárias
###
SELECT
    day,
    account_id,
    account_name,
    campaign_id,
    campaign_name,
    version,
    pool_id,
    pool_name,
    objective_id,
    objective_name,
    ROUND(objective_budget,3) as objective_budget,
    bid_strategy_type,
    subobjective_name,
    ROUND(subobjective_budget,3) as subobjective_budget,
    ROUND(subobjective_base_daily_budget,3) as subobjective_base_daily_budget,
    ROUND(subobjective_daily_budget,3) as subobjective_daily_budget,
    ROUND(subobjective_cost_share,3) as subobjective_cost_share,
    num_days,
    ROUND(cpa_over_window,3) as cpa_over_window,
    ROUND(cpc_over_window,3) as cpc_over_window,
    ROUND(cpm_over_window,3) as cpm_over_window,
    ROUND(cpv_over_window,3) as cpv_over_window,
    ROUND(roas_over_window,3) as roas_over_window,
    optimization_metric,
    ROUND(optimization_metric_value,3) as optimization_metric_value,
    ROUND(optimization_metric_coeficient,3) as optimization_metric_coeficient,
    optimization_metric_value_source
FROM
    campaign_performance_with_kpi_coeficient_over_window
ORDER BY account_id, campaign_id, day DESC
""", job_config=job_config)

query.result()

Abaixo, temos dados estatísticos do dataset a critério de análise.

In [18]:
# %%bigquery df --project $GOOGLE_BIGQUERY_PROJECT
# WITH
# ###
# # Realiza a soma das métricas brutas nos últimos @MOVING_AVARAGE_DAYS
# ###
# campaign_performance_over_window AS (
#   SELECT
#     *,
#     SUM(impressions)            OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS impressions_over_window,
#     SUM(clicks)                 OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS clicks_over_window,
#     SUM(video_views)            OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS video_views_over_window,
#     SUM(cost)                   OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS cost_over_window,
#     SUM(conversions)            OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS conversions_over_window,
#     SUM(conversion_value_sales) OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS conversion_value_sales_over_window,
#     COUNT(*)                    OVER(PARTITION BY campaign_id ORDER BY day ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS num_days
#   FROM
#     BUDGET_MANAGER_TEST.ggl_campaign_performance
#   WHERE
#     day >= DATE_SUB(DATE(DATE_SUB(CURRENT_DATE(), INTERVAL 2 DAY)), INTERVAL 14 DAY)
# ),

# ###
# # Realiza o cálculo dos kpis nos últimos @MOVING_AVARAGE_DAYS
# ###
# campaign_performance_with_kpis_over_window AS (
#   SELECT
#     *,
#     SAFE_DIVIDE(cost_over_window, conversions_over_window) AS cpa_over_window,
#     SAFE_DIVIDE(cost_over_window, clicks_over_window) AS cpc_over_window,
#     SAFE_DIVIDE(cost_over_window, impressions_over_window / 1000) AS cpm_over_window,
#     SAFE_DIVIDE(cost_over_window, video_views_over_window) AS cpv_over_window,
#     SAFE_DIVIDE(conversion_value_sales_over_window, cost_over_window) AS roas_over_window
#   FROM
#     campaign_performance_over_window
# ),

# ###
# # Realiza o cálculo do optimization_metric_value nos últimos @MOVING_AVARAGE_DAYS
# # particionado por dia
# ###
# campaign_performance_with_kpi_value_over_window AS (
#   SELECT
#     kd.day,
#     kd.account_id,
#     kd.account_name,
#     kd.campaign_id,
#     kd.campaign_name,
#     kd.version,
#     kd.pool_id,
#     kd.pool_name,
#     kd.objective_id,
#     kd.objective_name,
#     kd.objective_budget,
#     kd.bid_strategy_type,
#     kd.subobjective_name,
#     kd.subobjective_budget,
#     kd.subobjective_base_daily_budget,
#     kd.subobjective_daily_budget,
#     kd.subobjective_cost_share,
#     kd.last_month_objective_cost,
#     kd.last_month_subobjective_cost,
#     kd.this_month_subobjective_cost,
#     kd.optimization_metric,
#     num_days,
#     cpa_over_window,
#     cpc_over_window,
#     cpm_over_window,
#     cpv_over_window,
#     roas_over_window,
#     CASE
#         WHEN optimization_metric = "CPC" THEN cpc_over_window
#         WHEN optimization_metric = "CPA" THEN cpa_over_window
#         WHEN optimization_metric = "CPM" THEN cpm_over_window
#         WHEN optimization_metric = "CPV" THEN cpv_over_window
#         WHEN optimization_metric = "ROAS" THEN roas_over_window 
#     END optimization_metric_value,
#   FROM 
#     BUDGET_MANAGER_TEST.ggl_kpi_definition as kd
#   LEFT JOIN campaign_performance_with_kpis_over_window as kpiw
#   USING (account_id, campaign_id, day)
#   WHERE
#     day = DATE_SUB(CURRENT_DATE(), INTERVAL 2 DAY)
# ),

# ###
# # Realiza o cálculo do optimization_metric_coeficient nos últimos @MOVING_AVARAGE_DAYS
# ###
# campaign_performance_with_kpi_coeficient_over_window AS (
#   SELECT
#     *,
#     UTIL.get_optimization_coeficient_ggl(optimization_metric, optimization_metric_value) as optimization_metric_coeficient
#   FROM 
#     campaign_performance_with_kpi_value_over_window
# )

# ###
# # Filtra as colunas necessárias
# ###
# SELECT
#     day,
#     account_id,
#     account_name,
#     campaign_id,
#     campaign_name,
#     version,
#     pool_id,
#     pool_name,
#     objective_id,
#     objective_name,
#     ROUND(objective_budget,3) as objective_budget,
#     bid_strategy_type,
#     subobjective_name,
#     ROUND(subobjective_budget,3) as subobjective_budget,
#     ROUND(subobjective_base_daily_budget,3) as subobjective_base_daily_budget,
#     ROUND(subobjective_daily_budget,3) as subobjective_daily_budget,
#     ROUND(subobjective_cost_share,3) as subobjective_cost_share,
#     ROUND(last_month_objective_cost,3) as last_month_objective_cost,
#     ROUND(last_month_subobjective_cost,3) as last_month_subobjective_cost,
#     ROUND(this_month_subobjective_cost,3) as this_month_subobjective_cost,
#     num_days,
#     ROUND(cpa_over_window,3) as cpa_over_window,
#     ROUND(cpc_over_window,3) as cpc_over_window,
#     ROUND(cpm_over_window,3) as cpm_over_window,
#     ROUND(cpv_over_window,3) as cpv_over_window,
#     ROUND(roas_over_window,3) as roas_over_window,
#     optimization_metric,
#     ROUND(optimization_metric_value,3) as optimization_metric_value,
#     ROUND(optimization_metric_coeficient,3) as optimization_metric_coeficient
# FROM
#     campaign_performance_with_kpi_coeficient_over_window
# ORDER BY account_id, campaign_id, day DESC

In [19]:
# df.describe()

In [20]:
# df.head(100)